In [90]:
import numpy as np
import pandas as pd 
from numpy import nan
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_absolute_error
from correlation_pearson.code import CorrelationPearson
pearson = CorrelationPearson()
import operator

In [91]:
raw = pd.read_csv('demo_sidang.csv', delimiter=';')
# raw

In [93]:
# raw = raw[pd.notnull(raw['rating'])]
# raw = raw[pd.notnull(raw['timespent'])]
# raw
where_are_NaNs = np.isnan(raw)
raw[where_are_NaNs] = 0
# raw
pivot_raw = raw.pivot_table(index='user_id', columns='content_id', values='rating').fillna(0)
# pivot_raw

In [94]:
E = raw['rating']
a = raw['bookmarked']
t = raw['timespent']
c = raw['total_selection']
b = np.exp(-t)

In [95]:
normalize = lambda x : ((x - np.min(x))/(np.max(x)-np.min(x)))
A = normalize(a)
B = normalize(b)
C = normalize(c)

In [96]:
e = E
i = A + ((2)*(B)) + ((2)*(C)*(E))
S = (1/2)*(e+i)
# print(S)

In [97]:
user_id = raw[['user_id']]
content_id = raw[['content_id']]
d = {
    'A' : A, 'B' : B, 'C' : C,
    'implicit' : i, 'explicit' : e, 'S' : S
}
data = pd.DataFrame(data = d)

# data

In [98]:
joinraw = raw.join(data)
weight = joinraw[['user_id', 'content_id', 'S']]
# weight

In [99]:
llor = weight.pivot_table(index='user_id', columns='content_id', values='S').fillna(0)
# llor

In [100]:
llor_matrix = csr_matrix(llor.values)
llor_array = llor.values

In [101]:
dict_x={}
for i in range(len(llor_array)):
    dict_x[i]={}
    for j in range(len(llor_array)):
        if i==j:
            continue
        else:
            dict_x[i][j]= pearson.result(llor_array[i],llor_array[j])
    

In [153]:
dict_x={}
final_score=[]
final_seq=[]
k=200
for i,value_i in enumerate(list(llor.index)):
#     print("=========INI USER ID: ",value_i,"=================")
    dict_x[i]={}
    temp={}
    for j,value_j in enumerate(list(llor.index)):
        if i==j:
            continue
        else:
            temp[j]= pearson.result(llor_array[i],llor_array[j])
    tmp = {key: temp[key] if not np.isnan(temp[key]) else 0 for key in temp}
#     dict_x[i] = dict(sorted(tmp.items(), key=operator.itemgetter(1),reverse=True)[:10])
    tmp = dict(sorted(tmp.items(), key=operator.itemgetter(1),reverse=True)[:k])
    pearsonDF = pd.DataFrame.from_dict(tmp, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['user_id'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    mean_rating = [llor_array[y].mean() for y in list(pearsonDF['user_id'])]
    pearsonDF['ave_rating'] = mean_rating
#     print(pearsonDF)
    topUsersRating=pearsonDF.merge(weight, left_on='user_id', right_on='user_id', how='inner')
    topUsersRating['weight'] = topUsersRating['S'] - topUsersRating['ave_rating']
    topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['weight']
#     print(topUsersRating)
    tempTopUsersRating = topUsersRating.groupby('content_id').sum()[['similarityIndex','weightedRating']]
    tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
#     print(tempTopUsersRating)
    recommendation_df = pd.DataFrame()
    recommendation_df['recommendation score'] = llor_array[i].mean()+(tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex'])
    recommendation_df['content_id'] = tempTopUsersRating.index
#     print(recommendation_df)
    recommendation_df = recommendation_df.sort_values(by='recommendation score', ascending=False)
    for index, row in recommendation_df.iterrows():
        final_score.append([value_i,row['content_id'],row['recommendation score']])
    final_seq.append([value_i,list(recommendation_df["content_id"])])
#     print(recommendation_df)

In [154]:
final_score_df = pd.DataFrame(final_score,columns=["user_id","content_id","Recommendation Score"])
# final_score_df

In [155]:
llop = final_score_df.pivot_table(index='user_id', columns='content_id', values='Recommendation Score').fillna(0)
# llop

In [156]:
final_seq_df = pd.DataFrame(final_seq,columns=["user_id","Recommendation Sequence"])
# final_seq_df

In [157]:
y_true = pivot_raw.values
y_pred = llop.values
rating = np.reshape(y_true,(-1))
prediction = np.reshape(y_pred,(-1))
mean_absolute_error(rating, prediction)

5.619692989363384